In [1]:
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed

import networkx as nx
from sklearn.model_selection import train_test_split

c:\Users\thu71\AppData\Local\anaconda3\envs\pathml\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch_geometric.utils
import torch

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()

In [6]:
import lightning.pytorch as pl
import PPIGraph

c:\Users\thu71\AppData\Local\anaconda3\envs\pathml\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


# Load dataset

In [7]:
from torch_geometric.loader import DataLoader

# Define condition mapping
condition_mapping = {'HCC827Ctrl': 0, 'HCC827Osim': 1}

# Load graph dataset and process if neede
graph_path = data_dir / '9PPI' / 'graphs' 

dataset = PPIGraph.GraphDataset(graph_path, 'raw', 'pt', condition_mapping=condition_mapping, n_c=2)
train_set, val_set, test_set = PPIGraph.train_test_val_split(dataset)

# Create Dataloader
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)


In [8]:
# # Print some information on the dataset
# print(f'Dataset: {dataset}:')
# print('======================')
# print(f'Number of graphs: {len(dataset)}')
# print(f'Number of features: {dataset.num_features}')
# print(f'Number of classes: {dataset.num_classes}')

# print(f'Train set: {len(train_set)}, test set: {len(test_set)}, val set: {len(val_set)}')
# for step, data in enumerate(train_loader):

#     print(f'Step {step + 1}:')
#     print('=======')
#     print(f'Number of graphs in the current batch: {data.num_graphs}')
#     print(data)
#     print()
#     break

# K-fold on filter dataset

In [9]:
import itertools
from lightning.pytorch.accelerators import find_usable_cuda_devices
import wandb

In [10]:
from sklearn.model_selection import KFold, StratifiedKFold
from torch.utils.data import SubsetRandomSampler

In [11]:
# Filter out by maximum number of counts per cell
min_count = 100
max_count = 400

graph_path = data_dir / '9PPI' / 'graphs' 
dataset = PPIGraph.GraphDataset(graph_path, 'raw', 'pt', condition_mapping=condition_mapping, n_c=2)

# Create Dataloader
loader = DataLoader(dataset, batch_size=1, shuffle=False)

# Get Indices
indices = []
for step, data in enumerate(loader):
    if len(data.x) <= min_count:
        continue 
    
    if (data.x.sum(axis=0) >= max_count).any():
        continue
    indices.append(step)
    
# Get subset dataset
dataset_filtered = dataset.index_select(indices)


In [12]:
len(dataset_filtered)

1368

In [13]:
dataset_name = '9PPI_v3'
condition = 'Kfold'
checkpoint_folder = (Path().cwd().parents[0]).absolute() / 'data' / '9PPI' /"saved_models" / dataset_name / f"Graph_GNNs_{condition}" 
project_name = f'PLA_10152023_{dataset_name}_{condition}'

In [14]:
AVAIL_GPUS = [0]

# Setting the seed
pl.seed_everything(42)

NUM_LAYERS = 2
HIDDEN_CHANNELS = 16
pools = ['mean', 'max', 'sum', 'attention', 'attention2']
num_layers = [2,3,4]
hiddens = [16, 32, 64]


epochs = 80
model = 'MLP'

Seed set to 42


In [15]:
torch.set_float32_matmul_precision('medium')

In [16]:
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

for fold, (train_ids, valid_ids) in enumerate(kfold.split(dataset_filtered)):
    train_subset = dataset_filtered.index_select(train_ids.tolist())
    val_subset = dataset_filtered.index_select(valid_ids.tolist())
    for NUM_LAYERS, HIDDEN_CHANNELS, pool, in list(itertools.product(*[num_layers, hiddens, pools])):
            # Path to the folder where the pretrained models are saved
        CHECKPOINT_PATH = checkpoint_folder / f'MLP_{NUM_LAYERS}_{HIDDEN_CHANNELS}_onehot_{fold}' / pool
        CHECKPOINT_PATH.mkdir(parents=True, exist_ok=True)

        # Skip already trained kfold and pool
        checkpoint = CHECKPOINT_PATH / f"GraphLevel{model}" / f"GraphLevel{model}.ckpt" 
        if checkpoint.exists():
            print(checkpoint)
            continue

        # Run training
        run = wandb.init(project=project_name, name=f'{model}_{NUM_LAYERS}_{HIDDEN_CHANNELS}_onehot_{fold}', 
                        group=f'{model}_{pool}', 
                        # mode="disabled"
                        )
        PPIGraph.train_graph_classifier_kfold(model, 
                                                train_subset, 
                                                val_subset, 
                                                dataset, 
                                                CHECKPOINT_PATH, 
                                                AVAIL_GPUS, 
                                                in_channels=9,
                                                hidden_channels=HIDDEN_CHANNELS, 
                                                out_channels = HIDDEN_CHANNELS,
                                                num_layers=NUM_LAYERS, 
                                                epochs=epochs,
                                                embedding=False,
                                                batch_size=258,
                                                graph_pooling=pool)
        run.finish()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: thoomas. Use `wandb login --relogin` to force relogin


Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Seed set to 42
c:\Users\thu71\AppData\Local\anaconda3\envs\pathml\lib\site-packages\pytorch_lightning\loggers\wandb.py:389: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
c:\Users\thu71\AppData\Local\anaconda3\envs\pathml\lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:639: Checkpoint directory \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\23_PLA_revision\data\9PPI\saved_models\9PPI_v3\Graph_GNNs_Kfold\MLP_2_16_onehot_0\mean\GraphLevelMLP exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type             | Params
-------------------------------------------------
0 | x_embedding | Identity         | 0     
1 | model     

Using MLP


c:\Users\thu71\AppData\Local\anaconda3\envs\pathml\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
c:\Users\thu71\AppData\Local\anaconda3\envs\pathml\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.
c:\Users\thu71\AppData\Local\anaconda3\envs\pathml\lib\site-packages\lightning\pytorch\utilities\data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 2. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
c:\Users\thu71\AppData\Local\anaconda3\envs\pathml\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: T

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▄▄▄▄▄▄▄▄▄▅▅▇▇▇▇▇▇█▇▇██▇▇▇▇▇█▇█▇█████▇█▇
train_auc,▁▁▂▁▁▁▂▂▂▃▄▅▇▇▇▇▇▇█▇▇▇▇▇█▇█▇█▇█▇█████▇██
train_f1,▇▂▁▁▁▁▁▁▁▁▃▄▆▇▇█▇▇█▇▇█▇▇██▇▇█▇█▇█████▇█▇
train_loss_epoch,█▆▆▆▆▆▆▆▅▅▅▄▂▃▂▃▃▂▂▂▂▂▁▂▂▁▁▁▂▁▁▂▁▁▂▂▁▂▁▂
train_loss_step,▇▆█▇▇▇▇▇▆▆█▆▃▅▇▄▄▅▅▄▆▆▅▄▁▄█▃█▃▅▄▄▂▂▄▄▄▆▄
trainer/global_step,▁▁▁▁▂▂▁▁▂▃▂▂▃▃▃▂▄▄▄▄▅▅▅▅▃▅▆▆▃▃▆▇▃▃▇▇▄▄██
val_acc,▁▁▁▁▁▁▁▁▁▂▃▆▇▇▇▇▆█▆▆▇▇▆▆▇▇▆▇█▇█▇▇█▇█▇▇█▆
val_auc,▁▅▆▇▇██▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
val_f1,▁▁▁▁▁▁▁▁▁▂▃▅▆▇▆▆▇▇▇█▆▇▇▇▇▇▇▇█▆▇▇▇█▇▇▇▇▇▆


...

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▄▄▄▄▄▄▄▄▄▆▆▇▇█▇█▇▇▇▇▇▇█▇▇▇▇█▇█▇▆▇▇███▇▇
train_auc,▁▂▂▂▂▂▁▂▃▄▆▆▆▇▇▇▇▇▇▇▇▇▇█▇▇▇▇▇████▇▇███▇█
train_f1,▆▂▁▂▁▁▁▁▁▆▆▆▆▇▇▇▇▇█▇▇█▇▇▇▆▇▇█▇█▇▅▅▇▇█▇▆█
train_loss_epoch,█▇▇▆▇▆▇▇▇▆▃▄▃▃▃▂▂▃▃▃▂▄▂▁▂▂▃▃▂▂▁▂▄▂▃▂▂▁▂▃
train_loss_step,▆▇▇▆▇▇▇▆▇▆█▄▁▅▇▃▄▇▆▃▄▆▃█▄▆▃▅▄▅▆▃▂▅▇▄▆▇▆▃
trainer/global_step,▁▁▁▁▂▂▁▁▂▃▂▂▃▃▃▂▄▄▄▄▅▅▅▅▃▅▆▆▃▃▆▇▃▃▇▇▄▄██
val_acc,▁▁▁▁▁▁▁▁▁▁▅▄▇▆▆▆▆▆▄▇▇▄▅▇▇███▅▇▄▇▇▅▇▆▅▇▇▅
val_auc,▁▂▂▂▂▂▃▂▆█████████████████████▇████▇▇▇▇▇
val_f1,▁▁▁▁▁▁▁▁▁▁▄▄▆▅▆▅▆▆▄▆▆▄▅▆▆█▇▇▅▆▄▇▇▅▆▆▅▆█▅


Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Seed set to 42
c:\Users\thu71\AppData\Local\anaconda3\envs\pathml\lib\site-packages\pytorch_lightning\loggers\wandb.py:389: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
c:\Users\thu71\AppData\Local\anaconda3\envs\pathml\lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:639: Checkpoint directory \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\23_PLA_revision\data\9PPI\saved_models\9PPI_v3\Graph_GNNs_Kfold\MLP_4_64_onehot_4\attention2\GraphLevelMLP exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name        | Type             | Params
--------------------------------------------------
0  | x_embedding | Identity         | 0     
1  | 

Using MLP


c:\Users\thu71\AppData\Local\anaconda3\envs\pathml\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.
c:\Users\thu71\AppData\Local\anaconda3\envs\pathml\lib\site-packages\lightning\pytorch\loops\fit_loop.py:293: The number of training batches (5) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
`Trainer.fit` stopped: `max_epochs=80` reached.


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▃▄▄▄▄▄▄▄▆▇▇▇▆▆▆▇▇▇▇▇▇▇▇▇▇█▇▇█▇▇█▇██▆▇▇█
train_auc,▃▂▂▃▂▁▂▂▂▅▆▇▇▇▇▇▇▇█▇▇▇▇▇███▇▇▇▇▇███▇▇▇█▇
train_f1,▆▃▁▁▁▁▁▁▁▅▇█▇▅▅▆▇▆▇█▇▆▇▇▇▇████▇▇█▇▇▇▅▆▇▇
train_loss_epoch,█▇▇▇▇▇▇▇▇▆▅▄▄▄▄▄▃▄▃▄▃▃▃▃▃▂▃▃▃▂▃▂▂▁▃▂▄▃▂▃
train_loss_step,██▇▇█▆▇▇█▇▄▇▅▇▆▆▄▅▆▅▅▄▃▇▃▅▅▂▃▇▃▇▆▁▆▃▅▅▃▄
trainer/global_step,▁▁▁▁▂▂▁▁▂▃▂▂▃▃▃▂▄▄▄▄▅▅▅▅▃▅▆▆▃▃▆▇▃▃▇▇▄▄██
val_acc,▁▁▁▁▁▁▁▁▁▃▆▄▇▆▆▇▆▇▇▆██▇▇▇█▆▆▆▆█▇▇▇█▆▆▇▇█
val_auc,▁▂▃▂▄▁▅▄▆▇███████████▇▇▇▇▇▆▆▆▆▆▆▆▆▆▇▇▇▇▆
val_f1,▁▁▁▁▁▁▁▁▁▃▆▄▇█▆▇█▇▆▇▇▇▇▇▇▇▆▆▆▇▇▇▇▇▇▇▇▇▇▇
